In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# common
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")

In [ ]:
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')

In [ ]:
print('train:',train.shape,'test:',test.shape)

In [ ]:
full = train.append(test,ignore_index = True)
data = full.copy()
data.shape

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.describe(include='O')

In [ ]:
data[['Pclass','Survived']].groupby('Pclass',as_index = False).mean().sort_values(by='Survived',ascending=False)

In [ ]:
data[['Sex','Survived']].groupby('Sex',as_index = False).mean().sort_values(by='Survived',ascending=False)

In [ ]:
data[['Embarked','Survived']].groupby('Embarked',as_index = False).mean().sort_values(by='Survived',ascending=False)

In [ ]:
g = sns.FacetGrid(data,col='Survived')
g.map(plt.hist,'Age')

In [ ]:
g = sns.FacetGrid(data,col='Survived',row='Sex')
g.map(plt.hist,'Age')

In [ ]:
g = sns.FacetGrid(data,col='Survived')
g.map(plt.hist,'Fare',bins=10)

In [ ]:
data.isnull().sum()

In [ ]:
print('Proportion of missing value in Cabin column',round(data['Cabin'].isnull().sum()/(data['Cabin'].isnull().sum()+data['Cabin'].count()) * 100,2),'%')

In [ ]:
data = data.drop('Cabin',axis=1)
data['Embarked'] = data['Embarked'].fillna('S')
data['Fare'] = data['Fare'].fillna(data['Fare'].mean())
data['Age'] = data['Age'].fillna(data['Age'].mean())

In [ ]:
PclassDf = pd.get_dummies(data['Pclass'],prefix='Pclass')
PclassDf.head()
data = data.join(PclassDf)
data = data.drop('Pclass',axis=1)

In [ ]:
data['Title'] = data['Name'].str.extract('([A-Za-z]+)\.', expand=False)
data['Title'].value_counts()

In [ ]:
data['Title'] = data['Title'].replace(['Sir','Jonkheer','Don'],'Mr')
data['Title'] = data['Title'].replace(['Mlle','Lady','Dona'],'Miss')
data['Title'] = data['Title'].replace(['Countess','Mme','Ms'],'Mrs')
data['Title'] = data['Title'].replace(['Capt','Rev','Dr','Major','Col'],'Officer')

In [ ]:
data['Title'].value_counts()

In [ ]:
data['Title'].isnull().sum()

In [ ]:
data[['Title','Survived']].groupby('Title',as_index = False).mean().sort_values(by='Survived',ascending=False)
data = data.drop('Name',axis=1)

In [ ]:
data.head()

In [ ]:
TitleDf = pd.get_dummies(data['Title'],prefix = 'Title')
TitleDf.head()

In [ ]:
data = data.join(TitleDf)
data = data.drop('Title',axis=1)
data.head()

In [ ]:
data['Sex'] = data['Sex'].map({'male':1,'female':0})

In [ ]:
data.head()

In [ ]:
data['AgeBand'] = pd.cut(data['Age'], 5,labels = [1,2,3,4,5])
data[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

In [ ]:
data.head()

In [ ]:
data = data.drop('Age',axis=1)
AgeBandDf = pd.get_dummies(data['AgeBand'],prefix='AgeBand')
data = data.join(AgeBandDf)

In [ ]:
data['Family'] = data['SibSp'] + data['Parch'] + 1
data.head()

In [ ]:
data['Family'].value_counts()

In [ ]:
data['FamilySize'] = pd.cut(data['Family'],[0,1,3,11] ,labels = ['Single','Small','Large'])
data[data['Family'] ==11][['Family','FamilySize']]

In [ ]:
FamilySizeDF = pd.get_dummies(data['FamilySize'],prefix='FamilySize')
FamilySizeDF.head()

In [ ]:
data = data.join(FamilySizeDF)
data = data.drop(['SibSp','Parch','Family','FamilySize','AgeBand','Ticket'],axis=1)
data.head()

In [ ]:
EmbarkedDf = pd.get_dummies(data['Embarked'],prefix='Embarked')
data = data.join(EmbarkedDf)
data = data.drop('Embarked',axis=1)

In [ ]:
corrDf = data.corr() 
corrDf

In [ ]:
abs(corrDf['Survived']).sort_values(ascending =False)[1:]

In [ ]:
train = data.loc[0:890,:]
test = data.loc[891:,:]

In [ ]:
source_y = train['Survived']
source_X = train.drop(['Survived','PassengerId'],axis=1)
predict_X = test.drop(['Survived','PassengerId'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split 
train_X, test_X, train_y, test_y = train_test_split(source_X ,
                                                    source_y,
                                                    train_size=.8)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100)
forest.fit(train_X,train_y)

In [ ]:
forest.score(train_X, train_y)

In [ ]:
forest.score(test_X, test_y)

In [ ]:
predict_y = forest.predict(predict_X)
predict_y = predict_y.astype(int)
predDf = pd.DataFrame( 
    { 'PassengerId':  test['PassengerId'], 
     'Survived': predict_y } )
predDf.reset_index(drop=True)

In [ ]:
predDf.to_csv('titanic_pred.csv', index = False )